## MAC
Stands for **Message Authentication**, is a short array of bits that authenticates (receiver can verify that the message originated from the stated sender) a message. In order to generate a MAC, the sender needs a *message* and a *secret key*. In order to verify the MAC, the receiver requires the message and the same secret key.

Message digest doesn't provide authenticity since if a message is intercepted, the attacker can modify the message and regenerate digest from the modified message. Same is not possible with MAC, therefore MAC provides both:
- **integrity**
- **authentication**
  
MAC does not provide **non-repudiation** since whoever possese the secret key can generate MAC.

MAC is generated using a *MAC function*.

## HMAC
Stands for **Hash-based Message Authentication**. Uses a *HMAC function* to generate MAC. The function is defined as:  
`HMAC(K, message) = H(K' XOR opad || H(K' XOR ipad || message))`, where  
- `message`: the message to be authenticated
- `H`: hash function like `SHA-256`
- `K`: the secret key
- `K'`: block-sized key derived from K depending on the hash function’s internal block size, `B`. `B` for `SHA-256` for example is 1088 bytes.
- `opad`: outer padding, consisting of byte 0x5c repeated `B` times
- `ipad`: inner padding, consisting of byte 0x36 repeated `B` times
- `||`: concatenation

A complicated function like above is required to counter various attacks on HMAC, therefore a simple implementation like `HMAC(K, message) = H(K || message)` would not be secure.

HMAC function using a particular hash function is called by the notation `HMAC-<hash function>`, for example HMAC-SHA-256. An HMAC function produces an HMAC value of the same length as the hash length of the underlying
hash function.

### HMAC Security
Is defined by `min(K_bits , L)` where `K_bits` is the security strength of the secret key and `L` is the hash length of the hash function used. As an example, security of HMAC-SHA-256 using secret key of 256 bits is 256 bits. As the formula suggests, using secret key longer than hash length doesn't improve upon security very much.

### MAC and Encryption
Using a MAC with encryption can be achieved using the following schemes:
- *Encrypt-then-MAC*: plaintext is encrypted, then MAC is calculated on the ciphertext and sent together with the ciphertext. Receiver can verify authenticity without decrypting. Recommended.
- *Encypt-and-MAC*: plaintext is encrypted, however the MAC is calculated on the plaintext instead of the ciphertext. Receiver needs to decrypt contents before validating authenticity.
- *MAC-then-encrypt*: MAC is calculated on the plaintext. Then plaintext concatenated with MAC is encrypted. Receiver needs to decrypt contents before validating authenticity.

Some ciphers support *authenticated encryption modes* providing authentication as one of their core features and do not require separate authentication operations, such as running an HMAC function on plaintext or ciphertext. 

## OpenSSL
**Generate secret key**: 256 bits encryption key using rand command
```bash
$ openssl rand -hex 32
406ff43f213c4f42ad771b9958bb38c2e39173e2aede8664bc965e67fe02c288
```

**Generate HMAC:**
```bash
$ openssl dgst -sha-256 -mac HMAC -macopt hexkey:406ff43f213c4f42ad771b9958bb38c2e39173e2aede8664bc965e67fe02c288 hello.txt
HMAC-SHA2-256(hello.txt)= dd35a82ea26d792c1013e018e7f91dffaaacab6744aba91e0a5317b85d5c6e36
```

## Java  

In [ ]:
/**
 * Generates an HMAC-SHA256 signature for the given message
 *
 * @param message   for which HMAC is calculated
 * @param secretKey used for HMAC derivation
 * @return HMAC of the given message
 * @throws NoSuchAlgorithmException
 * @throws InvalidKeyException
 */
public static byte[] getHMAC(byte[] message, SecretKey secretKey) throws NoSuchAlgorithmException, InvalidKeyException {
    SecretKeySpec keySpec = new SecretKeySpec(secretKey.getEncoded(), "HmacSHA256");

    // Initialise MAC
    Mac mac = Mac.getInstance("HmacSHA256");
    mac.init(keySpec);

    return mac.doFinal(message);
}